# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [2]:
import glob
import os
import btrack
print('Btrack version', btrack.__version__)
import napari
print('Napari version', napari.__version__)
import numpy as np

from skimage.io import imread
from natsort import natsorted
from napari_animation import AnimationWidget
from tqdm import tqdm
import tifffile as tiff

Btrack version 0.4.1
Napari version 0.4.7


In [3]:
PATH = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks'

In [5]:
files = natsorted(glob.glob(os.path.join(PATH, '*.png')))
files
# sort the files numerically
#files = sorted(files, key=lambda f: int(f[len(os.path.join(PATH, 'labels_')):-4]))

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000001_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000002_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000003_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000004_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000005_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Mana

In [6]:
gfp_files = [file for file in files if 'channel001' in file]
rfp_files = [file for file in files if 'channel002' in file]

### method 1 - using a numpy array

In this example, each image from the timelapse is a 3D volume (32 x 1200 x 1200) and there are 10 timepoints

In [7]:
def segmentation_arr(files):
    """Segmentation as numpy array."""
    
    stack = []
    for filename in tqdm(files):
        img = imread(filename)
        stack.append(img)
    return np.stack(stack, axis=0)

In [ ]:
stack = segmentation_arr(files)

In [8]:
gfp_stack = segmentation_arr(gfp_files)
#rfp_stack = segmentation_arr(rfp_files)

100%|██████████| 1014/1014 [00:50<00:00, 20.00it/s]


Now we print out the shape of the stack (T, Z, Y, X):

In [ ]:
stack.shape

In [10]:
gfp_stack.shape

(1013, 1352, 1688)

### method 2 - using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format. Note that the generator produces a numpy array for each image...

In [21]:
def segmentation_generator(files):
    """Segmentation generator"""
    
    for filename in files:
        img = imread(filename)
        yield img

In [ ]:
generator = segmentation_generator(files)

In [22]:
gfp_generator = segmentation_generator(gfp_files)
rfp_generator = segmentation_generator(rfp_files)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x). Note that we can also use scikit-image `regionprops` to calculate properties for each object, using the `properties` keyword:

In [ ]:
obj_from_arr = btrack.utils.segmentation_to_objects(stack, properties=('area', ))#, scale=(2., 1.))

two channel option

In [11]:
gfp_obj_from_arr = btrack.utils.segmentation_to_objects(gfp_stack, properties=('area', ))#, scale=(2., 1.))
rfp_obj_from_arr = btrack.utils.segmentation_to_objects(rfp_stack, properties=('area', ))#, scale=(2., 1.))

[INFO][2021/05/19 12:10:09 PM] Localizing objects from segmentation...
[INFO][2021/05/19 12:11:09 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/19 12:11:12 PM] ...Found 551730 objects in 1013 frames.
[INFO][2021/05/19 12:11:12 PM] Localizing objects from segmentation...
[INFO][2021/05/19 12:11:22 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/19 12:11:22 PM] ...Found 10306 objects in 1014 frames.


In [ ]:
# inspect the first object
obj_from_arr[0]

from generator 

(previously had difficulty visualising the tracks made from generator)

In [ ]:
obj_from_generator = btrack.utils.segmentation_to_objects(
    generator, 
    properties = ('area', 'major_axis_length'), 
   # scale=(2., 1.)
)

two channel option

In [ ]:
gfp_generator

In [23]:
gfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    gfp_generator, 
    properties = ('area', 'major_axis_length'), 
    #scale=(2., 1.)
)
rfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    rfp_generator, 
    properties = ('area', 'major_axis_length'), 
    #scale=(2., 1.)
)

[INFO][2021/05/19 02:39:00 pm] Localizing objects from segmentation...
[INFO][2021/05/19 02:42:46 pm] Objects are of type: <class 'dict'>
[INFO][2021/05/19 02:42:50 pm] ...Found 551730 objects in 1013 frames.
[INFO][2021/05/19 02:42:50 pm] Localizing objects from segmentation...
[INFO][2021/05/19 02:43:54 pm] Objects are of type: <class 'dict'>
[INFO][2021/05/19 02:43:54 pm] ...Found 10306 objects in 1014 frames.


In [ ]:
# inspect the first object
obj_from_generator[0]

## run btrack with the objects

We will use the objects from the generator here.

In [25]:
# # initialise a tracker session using a context manager
# with btrack.BayesianTracker() as tracker:

#     # configure the tracker using a config file
#     tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/cell_config.json')
#     tracker.max_search_radius = 10

#     # append the objects to be tracked
#     tracker.append(gfp_obj_from_arr) #obj_from_generator)
#     #tracker.append(gfp_obj_from_generator)

#     # set the volume
#     tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

#     # track them (in interactive mode)
#     tracker.track_interactive(step_size=100)

#     # generate hypotheses and run the global optimizer
#     tracker.optimize()

#     tracker.export(os.path.join(os.path.dirname(PATH), 'gfp_tracking.h5'), obj_type='obj_type_1')

#     # get the tracks in a format for napari visualization
#     gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
    
#     gfp_tracks = tracker.tracks
    
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/cell_config.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    #tracker.append(rfp_obj_from_arr) #obj_from_generator)
    tracker.append(rfp_obj_from_generator)

    # set the volume
    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(os.path.dirname(PATH), 'rfp_tracking2.h5'), obj_type='obj_type_2')

    # get the tracks in a format for napari visualization
    rfp_data, rfp_properties, rfp_graph = tracker.to_napari(ndim=2)
    
    rfp_tracks = tracker.tracks

[INFO][2021/05/19 02:52:07 pm] Loaded btrack: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/05/19 02:52:07 pm] btrack (v0.4.1) library imported
[INFO][2021/05/19 02:52:07 pm] Setting max XYZ search radius to: 100
[INFO][2021/05/19 02:52:07 pm] Starting BayesianTracker session
[INFO][2021/05/19 02:52:07 pm] Loading configuration file: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/cell_config.json
[INFO][2021/05/19 02:52:07 pm] Loading motion model: b'cell_motion'
[INFO][2021/05/19 02:52:07 pm] Setting max XYZ search radius to: 10
[INFO][2021/05/19 02:52:07 pm] Objects are of type: <class 'list'>
[INFO][2021/05/19 02:52:07 pm] Set volume to ((0, 1688), (0, 1352), (-100000.0, 100000.0))
[INFO][2021/05/19 02:52:07 pm] Starting tracking... 
[INFO][2021/05/19 02:52:07 pm] Tracking objects in frames 0 to 99 (of 1014)...
[INFO][2021/05/19 02:52:07 pm]  - Timing (Bayesian updates: 0.03ms, Linking: 0.04ms)
[INFO][2021/05/19 02:52

# move tracking files up a directory

In [ ]:
for track_file in glob.glob(os.path.join(PATH, '*.h5')):
    os.rename(track_file, os.path.join(os.path.dirname(PATH),os.path.basename(track_file)))

# add image layers

### compile stacks if necessary

In [ ]:
### compile stacks first if necessary 
root_folder = os.path.dirname(PATH)
pos = os.path.basename(root_folder)
### make stack folder if necessary
output_dir = os.path.join(root_folder,pos+'_stacks')
os.mkdir(output_dir) if os.path.exists(output_dir) == False else print(output_dir, 'already exists')
### count number of channels
raw_file_dir = os.path.join(root_folder, pos+'_raw')
file_list = natsorted(os.listdir(raw_file_dir))
channels = list(set([fn.split('channel')[1][0:3] for fn in file_list]))
for channel in ['000', '001', '002']:#channels:
    channel_file_list = natsorted([fn for fn in os.listdir(raw_file_dir) if 'channel'+channel in fn])
    stack = []
    for file in tqdm(channel_file_list):
        file = os.path.join(raw_file_dir, file)
        tif = np.array(tiff.imread(file),dtype=np.uint8)
        ### crop??
#         tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
#         tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
        stack.append(tif)
    stack = np.stack(stack,axis=0)
    with tiff.TiffWriter(os.path.join(output_dir,'channel{}.tif'.format(channel)),imagej=True) as output_file:
        output_file.save(stack)
    print('channel',channel, 'saved out')
    

In [16]:
#image_path = output_dir#'/home/nathan/data/kraken/pcna/MK0000/Pos15/stacks'
image_path = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks'

(glob.glob(os.path.join(image_path, '*.tif')))


['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel002.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel000.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel001.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel003.tif']

In [17]:
irfp = imread( '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel003.tif')
rfp = imread('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel002.tif')

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

In [30]:
viewer = napari.Viewer()
# viewer.add_labels(gfp_stack)#, scale=(1., 1., 1.), name='Segmentation')
viewer.add_labels(rfp_stack)#, scale=(1., 1., 1.), name='Segmentation')
# viewer.add_tracks(gfp_data, properties=gfp_properties, graph=gfp_graph, name='GFP Tracks')
viewer.add_tracks(rfp_data, properties=rfp_properties, graph=rfp_graph, name='RFP Tracks')
# viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "magenta")
# viewer.add_image(irfp, name="irfp", blending = "additive", colormap = "cyan")
    
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')